In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import folium

# Configure pandas and matplotlib settings for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
plt.style.use('seaborn-darkgrid')


In [2]:
# Loading data from CSV files
aadt_lines_data = pd.read_csv('projectdata/Maryland_Annual_Average_Daily_Traffic_-_Annual_Average_Daily_Traffic_(SHA_Statewide_AADT_Lines).csv')
aadt_points_data = pd.read_csv('projectdata/Maryland_Annual_Average_Daily_Traffic_-_Annual_Average_Daily_Traffic_(SHA_Statewide_AADT_Points).csv')
pedestrian_injury_data = pd.read_csv('projectdata/SHIP_Pedestrian_Injury_Rate_on_Public_Roads_2009-2022_20240505.csv')

# Preview the data to ensure it's loaded correctly
print(aadt_lines_data.head())
print(aadt_points_data.head())
print(pedestrian_injury_data.head())


C:\Users\aquil\AppData\Local\Temp/ipykernel_8248/3165192050.py:2: DtypeWarning: Columns (98) have mixed types. Specify dtype option on import or set low_memory=False.
  aadt_lines_data = pd.read_csv('projectdata/Maryland_Annual_Average_Daily_Traffic_-_Annual_Average_Daily_Traffic_(SHA_Statewide_AADT_Lines).csv')


   OBJECTID  LOCATION_ID  COUNTY_ID   COUNTY_DESC  MUN_SORT MUNICIPALITY  \
0         1  S1998030065          3     Baltimore         0         None   
1         2  S2008030007          3     Baltimore         0         None   
2         3  S2011020910          2  Anne Arundel         0         None   
3         4        B0810          2  Anne Arundel         0         None   
4         5  S2000020143          2  Anne Arundel         0         None   

                                   ROADNAME ID_PREFIX  ID_RTE_NO MP_SUFFIX  \
0  Ramp 4 Fr Md 146 Nb To Ramp 5 (to Is695)        RP        695             
1    Ramp 4 Fr Is 83 Nb To Middletown Rd Eb        RP         83             
2                                    Elm Rd        SR          9             
3                           John Hanson Hwy        IS        595             
4  Ramp 2 Fr Ramp 7 (fr Md100 Wb) To Md 713        RP        100             

   ID_MP  BEGIN_SECTION  END_SECTION  \
0   0.03           0.00         0.

C:\Users\aquil\AppData\Local\Temp/ipykernel_8248/3165192050.py:3: DtypeWarning: Columns (100) have mixed types. Specify dtype option on import or set low_memory=False.
  aadt_points_data = pd.read_csv('projectdata/Maryland_Annual_Average_Daily_Traffic_-_Annual_Average_Daily_Traffic_(SHA_Statewide_AADT_Points).csv')


In [3]:
def load_data_from_api(url):
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data)

# Example API URLs
traffic_stops_url = 'https://opendata.maryland.gov/resource/tx73-47dk.json'
pedestrian_injury_url = 'https://opendata.maryland.gov/resource/8iqk-udpa.json'

# Load data from API
traffic_stops_data = load_data_from_api(traffic_stops_url)
pedestrian_injury_data = load_data_from_api(pedestrian_injury_url)


In [4]:
# Convert dates to datetime format, handle missing values, etc.
traffic_stops_data['date'] = pd.to_datetime(traffic_stops_data['date'], errors='coerce')
pedestrian_injury_data.dropna(inplace=True)  # Example of dropping missing values


KeyError: 'date'

In [5]:
# Example: Merging datasets on common columns
combined_data = pd.merge(traffic_stops_data, pedestrian_injury_data, on='date', how='outer')


KeyError: 'date'

In [ ]:
# Display statistical summaries, value counts, etc.
print(combined_data.describe())
sns.countplot(x='severity', data=combined_data)
plt.show()


In [ ]:
# Creating maps and charts
map = folium.Map(location=[38.9072, -76.8569], zoom_start=11)  # Central Maryland coordinates
# Add data points to map
for idx, row in combined_data.iterrows():
    folium.CircleMarker([row['latitude'], row['longitude']],
                        radius=5,
                        color='red',
                        fill=True,
                        fill_color='red').add_to(map)
map.save('traffic_accidents_map.html')
